In [27]:
import cv2
import numpy as np

dirname = '/home/fabricio/Dropbox/UTFPR/visao_computacional/background_subtractor_opencv/'
filename = dirname + 'input.mp4'

def processar(frame, metodo, kernel=None):
    fgmask = metodo.apply(frame)
    fgmask = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    return fgmask

def concatenar_saida(images): 
    return cv2.vconcat(
        [cv2.hconcat(image) for image in images]
    )

# Apenas para fazer a separação - Método Fixo
def pegarPrimeiroFrame():
    cap = cv2.VideoCapture(filename)
    cap.set(1, 0); # Primeiro Frame
    ok, frame = cap.read()
    cap.release()
    return cv2.resize(frame, (320,320))

def pegarAlgunsFrames():
    cap = cv2.VideoCapture(filename)
    frameIds = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=25)
    frames = []
    for fid in frameIds:
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ok, frame = cap.read()
        frames.append(frame)
    cap.release()
    return frames

video = cv2.VideoCapture(filename)
# Modelo Fixo
frame_sem_bolinha = pegarPrimeiroFrame()

# Modelo Médio
avgValue = np.float32(frame_sem_bolinha)

# 2 métodos estatísticos. O melhor foi o KNN
fgbg1 = cv2.createBackgroundSubtractorMOG2()
fgbg2 = cv2.createBackgroundSubtractorKNN()

while True:
    ok, frame = video.read()
    if frame is None:
        break

    # Redimensionar
    frame = cv2.resize(frame, (320,320))
    
    # Modelo Fixo
    modelo_fixo = cv2.subtract(frame, frame_sem_bolinha)
    
    # Modelo Médio
    cv2.accumulateWeighted(frame, avgValue, 0.02)
    modelo_medio = cv2.convertScaleAbs(avgValue)
    
    # Subtrair fundo utilizando os métodos
    images = [
        # Fixo
        [frame, modelo_fixo],
        # Médio
        [frame, modelo_medio],
        # Estatistico
#         [frame, processar(frame, fgbg1)],
        [processar(frame, fgbg1), processar(frame, fgbg2)],
    ]
    
    # Imprimir o resultado
    cv2.imshow('Frame', concatenar_saida(images))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows() 